# Modelo predictivo para Beta Bank

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, recall_score, f1_score, roc_auc_score, roc_curve
import numpy as np

In [2]:
data = pd.read_csv('Churn.csv')
display(data)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
data.rename(columns={'RowNumber': 'row_number', 'CustomerId': 'customer_id', 'Surname': 'surname', 'CreditScore': 'credit_score', 'Geography': 'geography', 'Gender': 'gender', 'Age': 'age', 'Tenure': 'tenure', 'Balance': 'balance', 'NumOfProducts': 'num_of_products', 'HasCrCard': 'has_cr_card', 'IsActiveMember': 'is_active_member', 'EstimatedSalary': 'estimated_salary', 'Exited': 'exited'}, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_number        10000 non-null  int64  
 1   customer_id       10000 non-null  int64  
 2   surname           10000 non-null  object 
 3   credit_score      10000 non-null  int64  
 4   geography         10000 non-null  object 
 5   gender            10000 non-null  object 
 6   age               10000 non-null  int64  
 7   tenure            9091 non-null   float64
 8   balance           10000 non-null  float64
 9   num_of_products   10000 non-null  int64  
 10  has_cr_card       10000 non-null  int64  
 11  is_active_member  10000 non-null  int64  
 12  estimated_salary  10000 non-null  float64
 13  exited            10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


## Preparación de Datos

In [5]:
data['tenure'].fillna(-1, inplace=True)
data.sample(10)

C:\Users\Flia\AppData\Local\Temp\ipykernel_6332\2723048582.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['tenure'].fillna(-1, inplace=True)


,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
668,669,15640825,Loyau,695,Spain,Male,46,3.0,122549.64,1,1,1,56297.85,0
1193,1194,15779947,Thomas,363,Spain,Female,28,6.0,146098.43,3,1,0,100615.14,1
8849,8850,15721433,Hixson,664,France,Female,38,4.0,74306.19,2,1,0,154395.56,0
2596,2597,15763665,Y?,833,France,Female,28,4.0,136674.51,2,0,0,5278.78,0
454,455,15726631,Hilton,758,France,Female,39,6.0,127357.76,1,0,1,56577.00,0
1505,1506,15798975,Doherty,606,Germany,Male,48,4.0,132403.56,1,0,0,36091.91,1
101,102,15743192,Hung,623,France,Female,44,6.0,0.00,2,0,0,167162.43,0
4576,4577,15770984,Fanucci,697,Spain,Female,40,7.0,130334.35,2,0,1,116951.10,0
214,215,15785542,Kornilova,572,Germany,Male,26,4.0,118287.01,2,0,0,60427.30,0
9497,9498,15617454,Ifeatu,684,France,Female,25,1.0,0.00,2,0,1,144978.47,0


Se cambian los datos Nulos de la columna 'tenure' por -1 haciendo referencia que se desconoce el período durante el cual ha madurado el depósito a plazo fijo de un cliente, posiblemente por algún error de extracción de datos. ¿Por qué no colocamos 0? Porque se puede interpretar que el período durante el cual ha madurado el depósito es menor a un año, por otra parte no hay valores en la columna iguales a 0 por lo que podemos concluir que no hay depósitos que han madurado menos de una año.

### Creación Variable Dummys

In [6]:
from sklearn.preprocessing import StandardScaler

# One-hot encoding para variables categóricas
data = pd.get_dummies(data, columns=['geography', 'gender'], drop_first=True)

# Selecciona las columnas que quieres usar como features
features = data.drop(['row_number', 'customer_id', 'surname', 'exited'], axis=1)
target = data['exited']

# Divide en train y validación
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345
)

# Normaliza las variables numéricas
scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_valid = scaler.transform(features_valid)

In [7]:
# Verifica las formas de los conjuntos de datos
features_train.shape, features_valid.shape, target_train.shape, target_valid.shape

((7500, 11), (2500, 11), (7500,), (2500,))

## Modelo de Regresión Logística

In [8]:
model_logi = LogisticRegression(random_state=12345,  class_weight='balanced', max_iter=1000, solver='liblinear')

param_grid_logi = {
    'C': np.logspace(-4, 4, 10),  
    'penalty': ['l1', 'l2']  
}

grid_search_logi = GridSearchCV(estimator=model_logi,
                                param_grid=param_grid_logi,
                                cv=5,  
                                scoring='f1', 
                                verbose=1,  
                                n_jobs=-1)  


grid_search_logi.fit(features_train, target_train)

# Mostrar los mejores parámetros y el mejor score
print(f"\nMejores parámetros para LogisticRegression: {grid_search_logi.best_params_}")
print(f"Mejor score (F1 en CV) para LogisticRegression: {grid_search_logi.best_score_:.4f}")

best_logi_model = grid_search_logi.best_estimator_

target_pred_valid_logi = best_logi_model.predict(features_valid)
print(f"\nExactitud en el conjunto de Validación para LogisticRegression: {accuracy_score(target_valid, target_pred_valid_logi):.4f}")
print("Reporte de Clasificación para el conjunto de Validación en LogisticRegression:")
print(classification_report(target_valid, target_pred_valid_logi))

Fitting 5 folds for each of 20 candidates, totalling 100 fits

Mejores parámetros para LogisticRegression: {'C': np.float64(0.046415888336127774), 'penalty': 'l1'}
Mejor score (F1 en CV) para LogisticRegression: 0.4876

Exactitud en el conjunto de Validación para LogisticRegression: 0.7072
Reporte de Clasificación para el conjunto de Validación en LogisticRegression:
              precision    recall  f1-score   support

           0       0.90      0.71      0.79      1965
           1       0.40      0.71      0.51       535

    accuracy                           0.71      2500
   macro avg       0.65      0.71      0.65      2500
weighted avg       0.79      0.71      0.73      2500



## Modelo Random Forest Clasifier

In [9]:
model_forest = RandomForestClassifier(random_state=12345, class_weight='balanced')

param_grid_forest = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search_forest = GridSearchCV(estimator=model_forest,
                                  param_grid=param_grid_forest,
                                  cv=5,
                                  scoring='f1',
                                  verbose=1,
                                  n_jobs=-1)


grid_search_forest.fit(features_train, target_train)

print(f"\nMejores parámetros para RandomForestClassifier: {grid_search_forest.best_params_}")
print(f"Mejor score (F1 en CV) para RandomForestClassifier: {grid_search_forest.best_score_:.4f}")


best_forest_model = grid_search_forest.best_estimator_

target_pred_val_forest = best_forest_model.predict(features_valid)
print(f"\nExactitud en el set de VALIDACIÓN para RandomForestClassifier: {accuracy_score(target_valid, target_pred_val_forest):.4f}")
print("Reporte de Clasificación en el set de VALIDACIÓN para RandomForestClassifier:")
print(classification_report(target_valid, target_pred_val_forest)) 

Fitting 5 folds for each of 81 candidates, totalling 405 fits

Mejores parámetros para RandomForestClassifier: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
Mejor score (F1 en CV) para RandomForestClassifier: 0.6238

Exactitud en el set de VALIDACIÓN para RandomForestClassifier: 0.8360
Reporte de Clasificación en el set de VALIDACIÓN para RandomForestClassifier:
              precision    recall  f1-score   support

           0       0.91      0.88      0.89      1965
           1       0.61      0.66      0.63       535

    accuracy                           0.84      2500
   macro avg       0.76      0.77      0.76      2500
weighted avg       0.84      0.84      0.84      2500



## Modelo Decision Tree Classifier

In [10]:
from sklearn.utils import resample
model_tree = DecisionTreeClassifier(random_state=12345, class_weight='balanced')


def sample(features, target):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_ones = features[target == 1]
    target_ones = target[target == 1]
    features_zeros = features[target == 0] # Majority
    target_zeros = target[target == 0]

    features_one_resampled, target_ones_resampled = resample(           # Suffle nos daba un rendimiento menor
        features_ones, target_ones,
        replace=True,
        n_samples=len(features_zeros),
        random_state=12345
    )
    features_balanced = pd.concat([features_zeros, features_one_resampled])
    target_balanced = pd.concat([target_zeros, target_ones_resampled])
    
    return features_balanced, target_balanced

features_train_df = pd.DataFrame(features_train, columns=features.columns)
features_valid_df = pd.DataFrame(features_valid, columns=features.columns)

features_train_df = features_train_df.reset_index(drop=True)
target_train_reset = target_train.reset_index(drop=True)

features_train_balanced, target_train_balanced = sample(features_train_df, target_train_reset)


param_grid_tree_simple = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

grid_search_tree = GridSearchCV(estimator=model_tree,
                                           param_grid=param_grid_tree_simple,
                                           cv=5,
                                           scoring='f1',
                                           verbose=1,
                                           n_jobs=-1)

grid_search_tree.fit(features_train_balanced, target_train_balanced)

print(f"\nMejores parámetros para DecisionTreeClassifier: {grid_search_tree.best_params_}")
print(f"Mejor F1 Score: {grid_search_tree.best_score_:.4f}")

best_tree_oversampled = grid_search_tree.best_estimator_
target_pred_val_tree_oversampled = best_tree_oversampled.predict(features_valid)

print(f"Exactitud en validación: {accuracy_score(target_valid, target_pred_val_tree_oversampled):.4f}")
print("Reporte de Clasificación:")
print(classification_report(target_valid, target_pred_val_tree_oversampled))

Fitting 5 folds for each of 72 candidates, totalling 360 fits

Mejores parámetros para DecisionTreeClassifier: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
Mejor F1 Score: 0.8381
Exactitud en validación: 0.7620
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.89      0.80      0.84      1965
           1       0.46      0.63      0.53       535

    accuracy                           0.76      2500
   macro avg       0.67      0.72      0.69      2500
weighted avg       0.80      0.76      0.77      2500



e:\Nico_repository\beta_bank_Sckitlearn\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [11]:
models = {
    "Logistic Regression": best_logi_model,
    "Random Forest": best_forest_model,
    "Decision Tree": best_tree_oversampled
}

for name, model in models.items():
    y_proba = model.predict_proba(features_valid)[:, 1]
    auc = roc_auc_score(target_valid, y_proba)
    f1 = f1_score(target_valid, model.predict(features_valid))
    print(f"{name} - F1: {f1:.4f} | AUC-ROC: {auc:.4f}")

# Evaluación final en el conjunto de prueba
# Primero, divide el conjunto de validación para obtener un conjunto de prueba
features_train_full, features_test, target_train_full, target_test = train_test_split(
    features, target, test_size=0.20, random_state=12345
)

# Entrena el mejor modelo (por ejemplo, el árbol de decisión) en todos los datos de entrenamiento
best_forest_model.fit(features_train_full, target_train_full)
y_pred_test = best_forest_model.predict(features_test)
y_proba_test = best_forest_model.predict_proba(features_test)[:, 1]

print("\nEvaluación final en el conjunto de prueba (Random Forest):")
print(f"F1: {f1_score(target_test, y_pred_test):.4f}")
print(f"AUC-ROC: {roc_auc_score(target_test, y_proba_test):.4f}")
print(classification_report(target_test, y_pred_test))

Logistic Regression - F1: 0.5094 | AUC-ROC: 0.7645
Random Forest - F1: 0.6333 | AUC-ROC: 0.8667
Decision Tree - F1: 0.5326 | AUC-ROC: 0.7759


e:\Nico_repository\beta_bank_Sckitlearn\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
e:\Nico_repository\beta_bank_Sckitlearn\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(



Evaluación final en el conjunto de prueba (Random Forest):
F1: 0.6379
AUC-ROC: 0.8698
              precision    recall  f1-score   support

           0       0.91      0.88      0.89      1573
           1       0.61      0.67      0.64       427

    accuracy                           0.84      2000
   macro avg       0.76      0.78      0.77      2000
weighted avg       0.84      0.84      0.84      2000



Se concluye que el modelo Random Forest Classifier es el más competente a la hora de calificar los modelos por medio de la métrica F1. Además, la métrica AUC-ROC nos indica que el modelo tiene un muy buen desempeño al momento de elegir quien se va de la empresa y quien no. Este modelo obtiene unas puntuaciónes de F1: 0.63 y AUC-ROC: 0.86, superiores a los modelos siguientes.